# Data Scraping(Poems of Bharathiyar):

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import random
from urllib.parse import urljoin
from concurrent.futures import ThreadPoolExecutor

In [21]:
# main index page of Bharathiyar Poems on TVA
index_url = "https://www.tamilvu.org/library/l9100/html/l9100ba1.htm"
base_url = "https://www.tamilvu.org/library/l9100/html/"

## Scraping the links of all the poems from Tamil Virtual Library

In [22]:
def scrape_bharathi_links():
    print("Connecting to Tamil Virtual Academy...")
    response = requests.get(index_url)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')

    poem_links = []
    
    # We look for all links that contain 'l9100pd1.jsp'
    # These are the actual poem pages
    for a_tag in soup.find_all('a', href=True):
        href = a_tag['href']
        
        if 'l9100pd1.jsp' in href:
            # These links start with /slet/, so we join them to the domain root
            full_url = urljoin(base_url, href)
            if full_url not in poem_links:
                poem_links.append(full_url)
    return poem_links

poem_links = scrape_bharathi_links()
print(f"Found {len(poem_links)} poem links.")
print("Sample links:", poem_links[:15] if poem_links else "No links found")

Connecting to Tamil Virtual Academy...
Found 331 poem links.
Sample links: ['https://www.tamilvu.org/slet/l9100/l9100pd1.jsp?bookid=145&pno=1', 'https://www.tamilvu.org/slet/l9100/l9100pd1.jsp?bookid=145&pno=2', 'https://www.tamilvu.org/slet/l9100/l9100pd1.jsp?bookid=145&pno=3', 'https://www.tamilvu.org/slet/l9100/l9100pd1.jsp?bookid=145&pno=4', 'https://www.tamilvu.org/slet/l9100/l9100pd1.jsp?bookid=145&pno=5', 'https://www.tamilvu.org/slet/l9100/l9100pd1.jsp?bookid=145&pno=6', 'https://www.tamilvu.org/slet/l9100/l9100pd1.jsp?bookid=145&pno=7', 'https://www.tamilvu.org/slet/l9100/l9100pd1.jsp?bookid=145&pno=8', 'https://www.tamilvu.org/slet/l9100/l9100pd1.jsp?bookid=145&pno=9', 'https://www.tamilvu.org/slet/l9100/l9100pd1.jsp?bookid=145&pno=10', 'https://www.tamilvu.org/slet/l9100/l9100pd1.jsp?bookid=145&pno=11', 'https://www.tamilvu.org/slet/l9100/l9100pd1.jsp?bookid=145&pno=12', 'https://www.tamilvu.org/slet/l9100/l9100pd1.jsp?bookid=145&pno=13', 'https://www.tamilvu.org/slet/l9100/

In [23]:
with open('bharathi_links.txt', 'w', encoding='utf8') as f:
    for link in poem_links:
        f.write(link + "\n")

## Scraping the poems from the link
Testing the logic with a single link.

In [24]:
test_link = "https://www.tamilvu.org/slet/l9100/l9100pd1.jsp?bookid=145&pno=2"

def test_single_scrape(url):
    print(f"Testing Scrape on: {url}\n" + "-"*30)
    
    response = requests.get(url)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')

    # 1. Test Title Extraction
    title_tag = soup.find('font', color="#990000")
    title = title_tag.get_text(strip=True) if title_tag else "Title Not Found"
    print(f"TITLE FOUND: {title}")

    # 2. Test Content Extraction
    print("\nCONTENT FOUND:")
    poem_lines = []
    
    # We look for the font tags used for poem text
    for font_tag in soup.find_all('font', face="GIST-TMOTChanakya"):
        text = font_tag.get_text(separator='\n', strip=True)
        
        # Skip the title if it repeats in the font tags
        if title in text:
            continue
            
        # Skip purely numeric verse numbers (like '1', '2', '3')
        if text.isdigit():
            continue
            
        poem_lines.append(text)
        print(f"\n{text}")

    if not poem_lines:
        print("FAILED: No poem lines extracted. Check the 'face' attribute in HTML.")

# Run the test
test_single_scrape(test_link)

Testing Scrape on: https://www.tamilvu.org/slet/l9100/l9100pd1.jsp?bookid=145&pno=2
------------------------------
TITLE FOUND: பாமாலை 
          : பக்தி பாடல்கள்

CONTENT FOUND:

தோத்திரப் பாடல்கள்
ஆறு துணை

ஓம்சக்தி ஓம்சக்தி ஓம் 
      -- பராசக்தி
ஓம்சக்தி ஓம்சக்தி ஓம்.
ஓம்சக்தி ஓம்சக்தி ஓம்சக்தி -- ஓம்சக்தி
ஓம்சக்தி ஓம்சக்தி ஓம்.

கணபதி ராயன் -- அவனிரு
காலைப் பிடித் திடுவோம்
குணமுயர்ந் திடவே -- விடுதலை
கூடி மகிழ்ந் திடவே

(
ஓம்சக்தி 
      ஓம்சக்தி ஓம்
)

சொல்லுக் கடங்காவே -- பராசக்தி
சூரத் தனங்க ளெல்லாம்;
வல்லமை தந்திடுவாள் -- பராசக்தி
வாழி யென்றே துதிப்போம்.

(
ஓம்சக்தி 
      ஓம்சக்தி ஓம்
)

வெற்றி வடிவேலன் -- அவனுடை
வீரத்தினைப் புகழ்வோம்;
சுற்றிநில் லாதேபோ! -- பகையே!
துள்ளி வருகுதுவேல்.

(
ஓம்சக்தி 
      ஓம்சக்தி ஓம்
)

தாமரைப் பூவினிலே -- சுருதியைத்
தனியிருந் துரைப்பாள்
பூமணித் தாளினையே -- கண்ணிலொற்றிப்
புண்ணிய மெய்திடுவோம்.

(
ஓம்சக்தி 
      ஓம்சக்தி ஓம்
)

பாம்புத் தலைமேலே -- நடஞ் செயும்
பாதத்தினைப் புகழ் வோம்
மாம்பழ வாயினிலே -- குழலிடஞ
வண்மை புகழ்ந்திடுவோம்.

(
ஓம்சக்தி 
 

In [ ]:
def scrape_bharathi_poems(poem_links):
    poems_results = []
    
    for idx, link in enumerate(poem_links):
        print(f"[{idx+1}/{len(poem_links)}] Scraping: {link}")
        try:
            response = requests.get(link)
            response.encoding = 'utf-8'
            soup = BeautifulSoup(response.text, 'html.parser')

            # Extract the title
            title_tag = soup.find('font', color="#990000")
            title = title_tag.get_text(strip=True) if title_tag else "Untitled"

            # extracting the poem
            poem_lines = []
            for font_tag in soup.find_all('font', face="GIST-TMOTChanakya"):
                text = font_tag.get_text(separator='\n', strip=True)
                
                # Filtering logic
                if title in text or text.isdigit():
                    continue
                
                poem_lines.append(text)

            content = "\n\n".join(poem_lines)

            # 3. Store in dictionary
            poems_results.append({
                'title': title,
                'content': content,
                'link': link
            })
            
        except Exception as e:
            print(f"Error on {link}: {e}")
        
        # Polite delay to avoid "Connection Reset" errors
        time.sleep(random.uniform(0.5, 1.0))
        
    return poems_results

In [27]:
# Use a session to keep the connection alive
session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
})

def scrape_worker(link):
    """The working logic wrapped for a thread worker"""
    try:
        # 15 second timeout is the 'sweet spot' for slow gov servers
        response = session.get(link, timeout=15)
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, 'html.parser')

        title_tag = soup.find('font', color="#990000")
        title = title_tag.get_text(strip=True) if title_tag else "Untitled"

        poem_lines = []
        for font_tag in soup.find_all('font', face="GIST-TMOTChanakya"):
            text = font_tag.get_text(separator='\n', strip=True)
            if title in text or text.isdigit():
                continue
            poem_lines.append(text)

        content = "\n\n".join(poem_lines)
        return {'title': title, 'content': content, 'link': link}
    except:
        return None

def fast_modular_scrape(poem_links):
    results = []
    # MAX_WORKERS = 3 is the magic number. 
    # It's fast, but doesn't trigger the "Attack Detected" alarm.
    with ThreadPoolExecutor(max_workers=3) as executor:
        # map() maintains the order of your links
        temp_results = list(executor.map(scrape_worker, poem_links))
    
    # Clean and Print Progress
    for idx, r in enumerate(temp_results):
        if r:
            results.append(r)
            # Print status every 25th as requested
            if (idx + 1) % 25 == 0:
                print(f"✅ Reached {idx+1} poems. Latest: {r['title']}")
                
    return results

In [ ]:
# --- EXECUTION ---
print("Starting Optimized Scrape (3x Speed)...")
final_data = fast_modular_scrape(poem_links)

# --- SAVING OUTSIDE ---
with open('input.txt', 'w', encoding='utf-8') as f:
    for poem in final_data:
        if poem['content'].strip():
            f.write(f"TITLE: {poem['title']}\n")
            f.write(poem['content'])
            f.write("\n\n\n")

print(f"Finished! Saved {len(final_data)} poems to input.txt")

Starting Optimized Scrape (3x Speed)...
✅ Reached 25 poems. Latest: பாமாலை 
          : பக்தி பாடல்கள்
✅ Reached 50 poems. Latest: பாமாலை 
          : பக்தி பாடல்கள்
✅ Reached 75 poems. Latest: பாமாலை 
          : பக்தி பாடல்கள்
✅ Reached 100 poems. Latest: தேசிய 
          கீதங்கள்
✅ Reached 125 poems. Latest: தேசிய 
          கீதங்கள்
✅ Reached 150 poems. Latest: தேசிய 
          கீதங்கள்
✅ Reached 175 poems. Latest: Untitled
✅ Reached 200 poems. Latest: Untitled
✅ Reached 225 poems. Latest: காவியங்கள் 
          : கற்பனையும் கதையும்
✅ Reached 250 poems. Latest: காவியங்கள் 
          : கற்பனையும் கதையும்
✅ Reached 275 poems. Latest: தனிப் 
          பாடல்கள் : பொதுமைப் பாடல்கள்
✅ Reached 300 poems. Latest: தனிப் 
          பாடல்கள் : பொதுமைப் பாடல்கள்
✅ Reached 325 poems. Latest: பிற்சேர்க்கை 
          : பல புதிய பாடல்கள்
Finished! Saved 331 poems to input.txt


In [28]:
with open('input.txt', 'r', encoding='utf8') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)

print(f"Total characters in dataset:{len(text)}")
print(f"Unique characters in dataset:{vocab_size}")

print(chars)


Total characters in dataset:438861
Unique characters in dataset:91
['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'E', 'I', 'L', 'T', 'U', '[', '\\', ']', '`', 'd', 'e', 'i', 'l', 'n', 't', '{', '}', '\xa0', 'ஃ', 'அ', 'ஆ', 'இ', 'ஈ', 'உ', 'ஊ', 'எ', 'ஏ', 'ஐ', 'ஒ', 'ஓ', 'க', 'ங', 'ச', 'ஜ', 'ஞ', 'ட', 'ண', 'த', 'ந', 'ன', 'ப', 'ம', 'ய', 'ர', 'ற', 'ல', 'ள', 'ழ', 'வ', 'ஷ', 'ஸ', 'ஹ', 'ா', 'ி', 'ீ', 'ு', 'ூ', 'ெ', 'ே', 'ை', 'ொ', 'ோ', 'ௌ', '்', '‘', '’', '“', '”']


## Cleaning the TVA dataset

In [33]:
import re

def clean_bharathi_dataset(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as f:
        text = f.read()

    # 1. Remove anything inside [ ] (Ragam, Thalam, Swara info)
    # The flags=re.DOTALL ensures it catches multi-line brackets
    text = re.sub(r'\[.*?\]', '', text, flags=re.DOTALL)

    # 2. Remove anything inside ( ) (Refrain instructions or repetitive markers)
    text = re.sub(r'\(.*?\)', '', text, flags=re.DOTALL)

    # 3. Remove "Source" lines (ஆதாரம் lines)
    # This looks for the word ஆதாரம் followed by anything until the end of the line
    text = re.sub(r'ஆதாரம்:.*', '', text)

    # 4. Remove standalone numbers followed by a dot (e.g., "15.")
    # We look for numbers at the start of a line
    text = re.sub(r'^\d+\.?\s*$', '', text, flags=re.MULTILINE)

    # 5. Clean up "Junk" whitespace
    # Replace 3 or more newlines with just 2 (standardizes poem spacing)
    text = re.sub(r'\n{3,}', '\n\n', text)
    
    text = re.sub(r'[a-zA-Z]', '', text)

    # Remove leading/trailing spaces from each line
    lines = [line.strip() for line in text.splitlines()]
    
    # Final pass: Remove lines that are just numbers or very short junk
    clean_text = "\n".join([l for l in lines if l.strip()])

    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(clean_text)

    print(f"Purification Complete!")
    print(f"Original size: {len(text)} chars")
    print(f"Cleaned size: {len(clean_text)} chars")

# Run it
clean_bharathi_dataset('input.txt', 'input_cleaned.txt')

Purification Complete!
Original size: 415763 chars
Cleaned size: 393882 chars


In [35]:
with open('input_cleaned.txt', 'r', encoding='utf8') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)

print(f"Total characters in dataset:{len(text)}")
print(f"Unique characters in dataset:{vocab_size}")

print(chars)

Total characters in dataset:393882
Unique characters in dataset:78
['\n', ' ', '!', '"', "'", ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '\\', ']', '`', '{', '}', '\xa0', 'ஃ', 'அ', 'ஆ', 'இ', 'ஈ', 'உ', 'ஊ', 'எ', 'ஏ', 'ஐ', 'ஒ', 'ஓ', 'க', 'ங', 'ச', 'ஜ', 'ஞ', 'ட', 'ண', 'த', 'ந', 'ன', 'ப', 'ம', 'ய', 'ர', 'ற', 'ல', 'ள', 'ழ', 'வ', 'ஷ', 'ஸ', 'ஹ', 'ா', 'ி', 'ீ', 'ு', 'ூ', 'ெ', 'ே', 'ை', 'ொ', 'ோ', 'ௌ', '்', '‘', '’', '“', '”']


### Note:
The text cleaned and kept is much worse than the one I copy pasted. Besides,  the data that i scraped is very less in density. It has only the third the size of tiny shakespeare dataset so it would make more sense to pre-train with a bigger dataset and then do fine tuning with the bharathiyar poems dataset.

# Pre-training

## Loading the Pre-training dataset and cleaning it

Data extracted from: [Wiki Articles](https://www.kaggle.com/datasets/aswin037/tamil-wiki-summarization?select=Tamil+wiki-data.csv)

In [15]:
import pandas as pd

# Load the CSV (using chunksize if your RAM is low, though 422MB should fit in RAM)
print("Reading CSV...")
df = pd.read_csv('tamil_wiki_data.csv')

# Look at the first few rows to find the right column name
print("Columns found:", df.columns.tolist())

# Assuming the column with the content is called 'text'
# We filter out any empty rows and English-only lines
with open('train_base.txt', 'w', encoding='utf-8') as f:
    for content in df['Text'].dropna():
        f.write(str(content) + "\n\n")

print("training text file created as train_base.txt")

Reading CSV...
Columns found: ['Text']
training text file created as train_base.txt


## Cleaning the dataset

In [ ]:
with open('train_base.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("Length of text in characters:",len(text))

Length of text in characters: 161205950


In [4]:
print(text[:1000])

டேனியல் பெர்ல்டேனியல் பெர்ல் (அக்டோபர் 10, 1963 – பிப்ரவரி 1, 2002) அமெரிக்க யூதப் பத்திரிகையாளராக இருந்தவர், இவர் அல்-கொய்தா தீவிரவாதிகளால் பாகிஸ்தான் கராச்சியில் கடத்தப்பட்டு, சித்திரவதைச் செய்யப்பட்டுக் கொலை செய்யப்பட்டார்.இவர் கடத்தப்பட்ட நேரத்தில், பெர்ல் "வால் ஸ்ட்ரீட் ஜர்னலின்" தெற்காசியச் செயலகத் தலைமையானவராகப் பணியாற்றினார், மேலும் இது இந்தியாவின், மகாராஷ்டிராவின், மும்பையைச் சார்ந்து இயங்கியது. அவர் ரிச்சர்ட் ரெய்ட் ("சூ பாம் வெடிப்பவர்"), அல்-கேடா மற்றும் பாகிஸ்தானின் உட்புற-சேவைகள் புலனாய்வு (ISI) ஆகியவற்றுக்கு இடையில் உள்ள குற்றஞ்சாட்டப்பட்ட தொடர்புகள் குறித்த விசாரணையின் ஒரு பகுதியாக பாகிஸ்தானுக்குச் சென்றிருந்தார். அதனைத் தொடர்ந்து அவர் அவரைக் கடத்தியவர்களால் தலை துண்டிக்கப்பட்டு மரணமடைந்தார்.ஜூலை 2002 இல், பாகிஸ்தானி வம்சாவளியைச் சேர்ந்த பிரிட்டிஷ் குடிமகனான அகமது ஓமர் சாயித் ஷேக்குக்கு பெர்லினைக் கடத்தியது மற்றும் கொலை செய்ததற்காக தூக்கு தண்டனை விதிக்கப்பட்டது, ஆனால் அவர் ஜனவரி 2010 வரை உயிருடன் இருந்தார்.மார்ச் 2007 இல், கியூபாவின், குவாண்டனமோ வளைகுடாவில் மூடப்பட்ட இர

In [5]:
type(text)

str

In [12]:
import re
import pandas as pd

def clean_wiki_text(text):
    if not isinstance(text, str): return ""

    # english letters
    text = re.sub(r'[a-zA-Z]', '', text)

    # multiple spaces and citations if any
    text = re.sub(r'\[\d+\]', '', text)

    # other languages
    clean_pattern = re.compile(r'[^0-9\u0b80-\u0bff\s\.,!\?\(\)\-\u200c\u200d]+')
    text = clean_pattern.sub('', text)

    # empty parantheses
    text = re.sub(r'\(\s*\)', '', text)
    text = re.sub(r'\[\s*\]', '', text)

    # new lines
    text = re.sub(r'\n{3,}', '\n\n', text)

    return text.strip()

with open('train_base1.txt', 'w', encoding='utf-8') as f:
    f.write(clean_wiki_text(text))

Load the cleaned dataset

In [13]:
with open('train_base1.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [14]:
print("Length of text in characters:",len(text))

Length of text in characters: 157808028


In [15]:
print(text[:1000])

டேனியல் பெர்ல்டேனியல் பெர்ல் (அக்டோபர் 10, 1963  பிப்ரவரி 1, 2002) அமெரிக்க யூதப் பத்திரிகையாளராக இருந்தவர், இவர் அல்-கொய்தா தீவிரவாதிகளால் பாகிஸ்தான் கராச்சியில் கடத்தப்பட்டு, சித்திரவதைச் செய்யப்பட்டுக் கொலை செய்யப்பட்டார்.இவர் கடத்தப்பட்ட நேரத்தில், பெர்ல் வால் ஸ்ட்ரீட் ஜர்னலின் தெற்காசியச் செயலகத் தலைமையானவராகப் பணியாற்றினார், மேலும் இது இந்தியாவின், மகாராஷ்டிராவின், மும்பையைச் சார்ந்து இயங்கியது. அவர் ரிச்சர்ட் ரெய்ட் (சூ பாம் வெடிப்பவர்), அல்-கேடா மற்றும் பாகிஸ்தானின் உட்புற-சேவைகள் புலனாய்வு  ஆகியவற்றுக்கு இடையில் உள்ள குற்றஞ்சாட்டப்பட்ட தொடர்புகள் குறித்த விசாரணையின் ஒரு பகுதியாக பாகிஸ்தானுக்குச் சென்றிருந்தார். அதனைத் தொடர்ந்து அவர் அவரைக் கடத்தியவர்களால் தலை துண்டிக்கப்பட்டு மரணமடைந்தார்.ஜூலை 2002 இல், பாகிஸ்தானி வம்சாவளியைச் சேர்ந்த பிரிட்டிஷ் குடிமகனான அகமது ஓமர் சாயித் ஷேக்குக்கு பெர்லினைக் கடத்தியது மற்றும் கொலை செய்ததற்காக தூக்கு தண்டனை விதிக்கப்பட்டது, ஆனால் அவர் ஜனவரி 2010 வரை உயிருடன் இருந்தார்.மார்ச் 2007 இல், கியூபாவின், குவாண்டனமோ வளைகுடாவில் மூடப்பட்ட இராணுவ விசார

In [16]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print('Vocab_size = ',vocab_size)


 !(),-.0123456789? ஂஃஅஆஇஈஉஊஎஏஐஒஓஔகஙசஜஞட஢ணதநனபமயரறலளழவஶஷஸஹாிீுூெேை௉ொோௌ்ௐௗ௦௧௨௩௪௫௬௭௮௯௰௱௲௳௴௵௶௹   ‌‍ 　
Vocab_size =  98


## Creating a mapping from characters to integers

stoi - string to integet  
itos - integer to string  
Creating an encoder and decoder for the conversion  

NOTE: Using byte-pair encoding and wordpiece was considered, but ultimately dissmissed as both would increase the vocabulary manifold and cause computational issues. At present it was decided that the computation was better used in having deeper networks rather than having a bigger embedding table.

In [17]:
stoi = { ch:i for i,ch in enumerate(chars)}
itos = { i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode('நான் வீழ்வேனென்று நினைத் தாயோ?'))
print(decode(encode('நான் வீழ்வேனென்று நினைத் தாயோ?')))

[43, 58, 44, 70, 1, 53, 60, 52, 70, 53, 64, 44, 63, 44, 70, 49, 61, 1, 43, 59, 44, 65, 42, 70, 1, 42, 58, 47, 68, 18]
நான் வீழ்வேனென்று நினைத் தாயோ?


## Encoding the entire dataset

In [18]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape,data.type)
print(data[:100])

torch.Size([157808028]) <built-in method type of Tensor object at 0x0000022CA9C98310>
tensor([39, 64, 44, 59, 47, 50, 70,  1, 45, 63, 48, 70, 50, 70, 39, 64, 44, 59,
        47, 50, 70,  1, 45, 63, 48, 70, 50, 70,  1,  3, 22, 34, 70, 39, 68, 45,
        48, 70,  1,  9,  8,  5,  1,  9, 17, 14, 11,  1,  1, 45, 59, 45, 70, 48,
        53, 48, 59,  1,  9,  5,  1, 10,  8,  8, 10,  4,  1, 22, 46, 63, 48, 59,
        34, 70, 34,  1, 47, 62, 42, 45, 70,  1, 45, 42, 70, 42, 59, 48, 59, 34,
        65, 47, 58, 51, 48, 58, 34,  1, 24, 48])


## Train test split

In [ ]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]